## 

# <font face='nexa' size=7> Working with Data <font size=4> (Basically Beautiful) </font>

## 

In [1]:
import pandas as pd
import numpy as np

In [89]:
population = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')

In [90]:
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')

In [91]:
abbv = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')

In [12]:
display(population.head(), population.shape)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


(2544, 4)

In [11]:
display(areas.head(), areas.shape)

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


(52, 2)

In [10]:
display(abbv.head(), abbv.shape)

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


(51, 2)

### Merging Population (which has only abbv) with abbv to get state names

In [87]:
pop_with_names = pd.merge(population, abbv, left_on= 'state/region', right_on= 'abbreviation')\
                                                                                        .drop('abbreviation', axis=1)

In [50]:
pop_with_names

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2443,WY,under18,1993,137458.0,Wyoming
2444,WY,total,1991,459260.0,Wyoming
2445,WY,under18,1991,136720.0,Wyoming
2446,WY,under18,1990,136078.0,Wyoming


# Step 1.  Length or Rows

    After merging two DFs, you must check if everything is in place or not.
    
    Like - Are there all rows coming?

In [55]:
population.shape[0]                                                                            #Initial Data Rows

2544

In [56]:
pop_with_names.shape[0]                                                                       #After merge Rows

2448

    So, no.
    Here as we have to explicitely gave the 'how', it will perform inner join.
    
    Now, giving it an Outer join to get every signle row. ( If your focus is 
                                                            on the left side
                                                            or the initial 
                                                            data rows then
                                                            left outer join
                                                            or
                                                            just outer join
                                                            will work. )

In [92]:
pop_with_names = pd.merge(population, abbv, how= 'left', left_on= 'state/region', right_on= 'abbreviation')\
                                                                                             .drop('abbreviation',1)

In [93]:
pop_with_names

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


Now check the length of rows

In [60]:
population.shape[0]                                                                            #Initial Data Rows

2544

In [61]:
pop_with_names.shape[0]                                                                       #After merge Rows

2544

# Step 2. Fill the missing data

    If you needed to perform the Step 1, then that means you have some missing data from either DFs.
    and even
    If you need not to perform the Step 1, then this also this step is essential


### *2.1 - Find WHERE?*

In [63]:
pop_with_names.isnull()                           #Just this, won't work...

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2539,False,False,False,False,True
2540,False,False,False,False,True
2541,False,False,False,False,True
2542,False,False,False,False,True


In [64]:
pop_with_names.isnull().any()                   #Now, got categorically!

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

#### Interpretation for 2.1 :
    
    So, from merging the data I found that in the second DF (right) where we were mapping the full state names
        are missing 
        
        and
        
        from the first DF (left) which is our basic data in which we had our population is missing somewhere.
        
##### Let's see for the ABBV first and then we will take over the population data.
        

### *2.2 Get WHERE?*

In [69]:
# Looking for the missing abbvs.
pop_with_names[ pop_with_names['state'].isnull() ]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


#### Interpretation for 2.2 :

    Just with the amazing masking, we got to know that the 'state' names (from the abbv - right df) are missing in
    the 96 rows. 
    
    ( Don't observe the NaN in population as we will take care of that later )
##### Let's fix the data

### *2.3 Fix WHERE?*

In [72]:
# It looks like there are some ABBVs from the left DF where we could not find the match in the right
# Which are they?

#                                                  'Because that was the key when we merged two DFs!'
#                                                  'state/region'... why?
pop_with_names[ pop_with_names['state'].isnull() ]['state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [96]:
#Fixing

pop_with_names.loc[ pop_with_names['state/region'] == 'PR', 'state'] = 'Puerto Rico'
pop_with_names.loc[ pop_with_names['state/region'] == 'USA', 'state'] = 'USA'

In [97]:
pop_with_names[ pop_with_names['state/region'] == 'PR' ].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,Puerto Rico
2449,PR,total,1990,NaN,Puerto Rico
2450,PR,total,1991,NaN,Puerto Rico
2451,PR,under18,1991,NaN,Puerto Rico
2452,PR,total,1993,NaN,Puerto Rico


In [98]:
pop_with_names[ pop_with_names['state/region'] == 'USA' ].head()

,state/region,ages,year,population,state
2496,USA,under18,1990,64218512.0,USA
2497,USA,total,1990,249622814.0,USA
2498,USA,total,1991,252980942.0,USA
2499,USA,under18,1991,65313018.0,USA
2500,USA,under18,1992,66509177.0,USA


#### Interpretation for 2.3 :

    Just assigned the realavent names where it was NaN before.
        You may say, hey! it is not something we want, but no. We are actually doing right, we SHOULD not 
        change the basic data to fix the current merge!
        
##### and NOTICE: I have to use .loc[] to assign the data.

    Where I could use:
    
        ✖    >>>       df[ df[col_to_find] == 'baseonValue' ]['col_to_change'] = 'New Value'
        
        ✔    >>>   df.loc[ df[col_to_find] == 'baseonValue',  'col_to_change'] = 'New Value'
        
        

#### 

### And we are done with the second step.

# Step 3. Repeat the Step 2 untill no column is with NaN value

Summerize:

    2.1 : Find WHERE
    2.2 : Get WHERE
    2.3 : Fix WHERE
    
##### 2.1

In [99]:
pop_with_names.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

##### 2.2

In [101]:
pop_with_names[ pop_with_names['population'].isnull() ]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,Puerto Rico
2449,PR,total,1990,NaN,Puerto Rico
2450,PR,total,1991,NaN,Puerto Rico
2451,PR,under18,1991,NaN,Puerto Rico
2452,PR,total,1993,NaN,Puerto Rico
2453,PR,under18,1993,NaN,Puerto Rico
2454,PR,under18,1992,NaN,Puerto Rico
2455,PR,total,1992,NaN,Puerto Rico
2456,PR,under18,1994,NaN,Puerto Rico
2457,PR,total,1994,NaN,Puerto Rico


##### We will not procceed ahead as the Data is missing from the Initial Dataset, and that data can't be filled randomly.
Hence, no ***2.3***

###  

# Step 4. Are you done?

    Not, here. Because we still need to add the Area column from the area dataset.
    Let's do that now.
    
**Four Steps ended and the work is done.**
If you still have another column to add, repeat Steps 1 to 4.

*(we have, that's why we will repeat them)*

In [103]:
pop_with_names.head()                              # Left 

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [104]:
areas.head()                                      # Right

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


Now, based **on *state*** we will do our **merge**.

In [107]:
finalDF = pd.merge(pop_with_names, areas)
finalDF

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423
1,AL,total,2012,4817528.0,Alabama,52423
2,AL,under18,2010,1130966.0,Alabama,52423
3,AL,total,2010,4785570.0,Alabama,52423
4,AL,under18,2011,1125763.0,Alabama,52423
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515
2492,PR,under18,2011,869327.0,Puerto Rico,3515
2493,PR,total,2011,3686580.0,Puerto Rico,3515
2494,PR,under18,2012,841740.0,Puerto Rico,3515


In [109]:
# Step.1 All Rows are there?
print(pop_with_names.shape[0])
print(finalDF.shape[0])

2544
2496


In [135]:
finalDF = pd.merge(pop_with_names, areas, how='left')
finalDF

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,USA,NaN
2540,USA,under18,2011,73902222.0,USA,NaN
2541,USA,total,2011,311582564.0,USA,NaN
2542,USA,under18,2012,73708179.0,USA,NaN


In [136]:
print(pop_with_names.shape[0])
print(finalDF.shape[0])

2544
2544


In [137]:
# Step 2. Fill the missing data

In [138]:
# 2.1 Find WHERE
finalDF.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

##### Population data can't be filled
##### Skipping that and going ahead to fix 'area (sq. mi)' if we can. Let's follow the sub-steps.

In [139]:
# 2.2 Get WHERE
finalDF[ finalDF['area (sq. mi)'].isnull() ]

,state/region,ages,year,population,state,area (sq. mi)
2496,USA,under18,1990,64218512.0,USA,NaN
2497,USA,total,1990,249622814.0,USA,NaN
2498,USA,total,1991,252980942.0,USA,NaN
2499,USA,under18,1991,65313018.0,USA,NaN
2500,USA,under18,1992,66509177.0,USA,NaN
2501,USA,total,1992,256514231.0,USA,NaN
2502,USA,total,1993,259918595.0,USA,NaN
2503,USA,under18,1993,67594938.0,USA,NaN
2504,USA,under18,1994,68640936.0,USA,NaN
2505,USA,total,1994,263125826.0,USA,NaN


In [140]:
# 2.3 Fix WHERE
finalDF[ finalDF['area (sq. mi)'].isnull() ]['state'].unique()

array(['USA'], dtype=object)

In [141]:
finalDF.shape

(2544, 6)

In [142]:
finalDF[finalDF['state'] == 'USA'].shape

(48, 6)

In [148]:
finalDF.dropna(inplace= True)

    Here, instead of fixing the data in the state of USA we just moved on and dropped EVERY rows with NaN
    which will also drop the rows with NaN in Population which we left before.

In [149]:
finalDF

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


In [175]:
# Step 3. Any missing data?
# Step 4. Are you done?

###  

###  

# Now,
As we have completed the work of building a single DF from the multiple datasets, let's get some insights from the data.

### Insight 1:
#### I want data where `year is 2010` and `agegroup is Total`

In [176]:
finalDF

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


In [207]:
Data2010 = finalDF[ (finalDF['year'] == 2010) & (finalDF['ages'] == 'total') ]
Data2010

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0
283,CO,total,2010,5048196.0,Colorado,104100.0
293,CT,total,2010,3579210.0,Connecticut,5544.0
379,DE,total,2010,899711.0,Delaware,1954.0
389,DC,total,2010,605125.0,District of Columbia,68.0
475,FL,total,2010,18846054.0,Florida,65758.0


### Insight 2:

#### I want the `Density` top to bottom

In [197]:
density = Data2010['population'] / Data2010['area (sq. mi)']
density

3         91.287603
91         1.087509
101       56.214497
189       54.948667
197      228.051342
283       48.493718
293      645.600649
379      460.445752
389     8898.897059
475      286.597129
485      163.409902
570      124.746707
581       18.794338
666      221.687472
677      178.197831
762       54.202751
773       34.745266
858      107.586994
869       87.676099
954       37.509990
965      466.445797
1050     621.815538
1061     102.015794
1146      61.078373
1157      61.321530
1242      86.015622
1253       6.736171
1338      23.654153
1349      24.448796
1434     140.799273
1445    1009.253268
1530      16.982737
1541     356.094135
1626     177.617157
1637       9.537565
1722     257.549634
1733      53.778278
1818      39.001565
1829     275.966651
1914     681.339159
1925     144.854594
2010      10.583512
2021     150.825298
2106      93.987655
2117      32.677188
2202      65.085075
2213     187.622273
2298      94.557817
2309      76.519582
2394      86.851900


In [196]:
density.sort_values(ascending= False, inplace= True)
density

389     8898.897059
2490    1058.665149
1445    1009.253268
1914     681.339159
293      645.600649
1050     621.815538
965      466.445797
379      460.445752
1541     356.094135
475      286.597129
1829     275.966651
1722     257.549634
197      228.051342
666      221.687472
2213     187.622273
677      178.197831
1626     177.617157
485      163.409902
2021     150.825298
1925     144.854594
1434     140.799273
570      124.746707
858      107.586994
1061     102.015794
2298      94.557817
2106      93.987655
3         91.287603
869       87.676099
2394      86.851900
1242      86.015622
2309      76.519582
2202      65.085075
1157      61.321530
1146      61.078373
101       56.214497
189       54.948667
762       54.202751
1733      53.778278
283       48.493718
1818      39.001565
954       37.509990
773       34.745266
2117      32.677188
1349      24.448796
1338      23.654153
581       18.794338
1530      16.982737
2010      10.583512
1637       9.537565
1253       6.736171


### Here we have the result, but with indexes. Which does not make any sense!

We may use some join or merge by index in density and Data2010 but then still we would have to access names with the index.

It would be better to set the index as the names of the states and then perform the operation.
#### Like this

In [204]:
Data2010.head(3)

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0


In [208]:
Data2010.set_index(['state'], inplace= True)
Data2010.head(3)

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0


In [213]:
density = Data2010['population'] / Data2010['area (sq. mi)']
density.sort_values(ascending= False).head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64